In [9]:
# General Imports
import numpy as np
import pandas as pd
from collections import Counter

# Visualisation Imports
import matplotlib.pyplot as plt

# Scikit Imports
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.datasets import load_iris

# Qiskit Imports
from qiskit import Aer, execute
from qiskit import BasicAer
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel
from qiskit.algorithms.optimizers import SPSA
from qiskit_machine_learning.algorithms import VQC
from qiskit.algorithms.optimizers import COBYLA
from qiskit.utils import QuantumInstance, algorithm_globals


In [2]:
df = load_iris()

In [3]:
#Iris Dataset
data = pd.DataFrame(data=df, columns=df.feature_names)
data.columns = ['feature{}'.format(i) for i in range(0,data.shape[1])]

dataset = pd.DataFrame(df.data).assign(target=df.target)
print (dataset.shape,pd.DataFrame(df.target).nunique().tolist()[-1:])

(150, 5) [3]


In [4]:
dataset_0 = dataset.copy()
dataset_0['target']= (dataset_0['target']==1)
dataset_0 = dataset_0.drop(dataset_0[dataset_0['target'] == False].sample(frac=0.80).index)
Counter(dataset_0['target'])

Counter({False: 20, True: 50})

In [5]:
X = np.array(dataset_0.iloc[: , :-1])
y = np.array(dataset_0.iloc[: , -1:].astype(int))


In [6]:
# Split dataset
sample_train, sample_test, label_train, label_test = train_test_split(
     X, y, test_size=0.2, random_state=22)

# Reduce dimensions (the dataset now has a dimensionality of 31 attributes)
n_dim = 4 #number of qubits we want to use
pca = PCA(n_components=n_dim).fit(sample_train)
sample_train = pca.transform(sample_train)
sample_test = pca.transform(sample_test)

# Normalise
std_scale = StandardScaler().fit(sample_train)
sample_train = std_scale.transform(sample_train)
sample_test = std_scale.transform(sample_test)

# Scale
samples = np.append(sample_train, sample_test, axis=0)
minmax_scale = MinMaxScaler((-1, 1)).fit(samples)
sample_train = minmax_scale.transform(sample_train)
sample_test = minmax_scale.transform(sample_test)

# Select
train_size = 100
sample_train = sample_train[:train_size]
label_train = label_train[:train_size]

test_size = 20
sample_test = sample_test[:test_size]
label_test = label_test[:test_size]

In [11]:
seed = 1024

feature_map = ZZFeatureMap(feature_dimension=n_dim, reps=2, entanglement="linear")
ansatz = TwoLocal(feature_map.num_qubits, ['ry', 'rz'], 'cz', reps=3)
vqc = VQC(feature_map=feature_map,
          ansatz=ansatz,
          optimizer=COBYLA(maxiter=100),
          quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
                                           shots=8192,
                                           seed_simulator=seed,
                                           seed_transpiler=seed)
          )
vqc.fit(sample_train, label_train)

score = vqc.score(sample_test, label_test)
print(f"Testing accuracy: {score:0.2f}")

/tmp/ipykernel_309/1847008678.py:8: DeprecationWarning: The class ``qiskit.utils.quantum_instance.QuantumInstance`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/qi_migration.
  quantum_instance=QuantumInstance(BasicAer.get_backend('statevector_simulator'),
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Testing accuracy: 0.86
